In [ ]:
# 读取csv
import csv
data = []
with open('../Data/squ2011_88.csv','rt') as f:
    csv_reader = csv.reader(f)
    isHeader = True                 # 判断帧头
    for row in csv_reader:          # 将csv 文件中的数据保存到data中
        if isHeader:
            isHeader = False
            continue
        frame = {
            'time':row[0],
            'xyd':row[1:4],
            'press':row[4],
            'mag4':row[5:17]
            }
        data.append(frame)           # 将字典加入到data数组中
print("read csv file successfully")

In [ ]:
# 获取原始数据
import numpy as np

length = len(data)
# 获取磁场数据，触点水平位置数据和压力数据
magnetic = []
position = []
press = []
for i,frame in enumerate(data):
    if(True):
        press.append(float(frame['press']))
        m = list(map(float, frame['mag4']))
        mm = [0,0,0,0,0,0,0,0,0,0,0,0]
        for k in range(3):
            for r in range(4):
                mm[r*3+k] = 2*m[k+r*3] - m[(k+1+r*3)%12] - m[k-1+r*3]
        magnetic.append(mm)
        position.append(list(map(float, frame['xyd'][:2])))

press = np.array(press)
magnetic = np.array(magnetic)
position = np.array(position)

In [ ]:
# 数据归一化
from sklearn.preprocessing import MinMaxScaler
# 创建 MinMaxScaler 对象
scaler = MinMaxScaler()
# 对触点位置与磁场数据进行 fit_transform 操作
normalized_magnetic = scaler.fit_transform(magnetic)

normalized_position = scaler.fit_transform(position)
normalized_press = (press - np.min(press))/(np.max(press) - np.min(press))
print(np.max(press) - np.min(press))

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Dropout

# 建立 CNN 模型
model = Sequential()
# 卷积层1，卷积核大小为3，激活函数为relu
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(12, 1)))
# 池化层1
model.add(MaxPooling1D(pool_size=2))
# 卷积层2，卷积核大小为3，激活函数为relu
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# 池化层2
model.add(MaxPooling1D(pool_size=2))
# 将数据展平成一维向量
model.add(Flatten())
model.add(Dense(64))
model.add(Dropout(0.5))
model.add(Dense(2, activation='linear'))

model.summary()


In [ ]:
from sklearn.model_selection import train_test_split
import keras

# 划分训练集和测试集
train_x, test_x, train_y, test_y = train_test_split(
    normalized_magnetic, position, test_size = 0.20, random_state = 42)

# model = keras.models.load_model("./cnn_server/stm32.h5")
# 编译模型
model.compile(loss='mse', 
              optimizer='adam',
              metrics='mae')
# 训练模型
history = model.fit(
    train_x, 
    train_y, 
    sample_weight = None,
    epochs=100, 
    batch_size=64, 
    verbose=2,
    validation_data=(test_x, test_y))
# model.save("./position_model_with_weight.h5")

In [ ]:
import matplotlib.pyplot as plt

epochs=range(len(history.history['mae']))
plt.figure()
plt.plot(epochs,history.history['mae'],'b',label='Training MAE')
plt.plot(epochs,history.history['val_mae'],'r',label='Validation MAE')
plt.title('Traing and Validation Mean Absolute Error')
plt.legend()

plt.figure()
plt.plot(epochs,history.history['loss'],'b',label='Training loss')
plt.plot(epochs,history.history['val_loss'],'r',label='Validation val_loss')
plt.title('Traing and Validation loss')
plt.legend()
